# Policy GAP Analyzer

> The input to this API is a zipped file containing Standard's JSON and Policy_JSON. The API splits the controls from the standard and then compares each control against the given policy. This allows the system to generate a gap analysis report, which is output as a ZIP (GAP_JSON) file. 

In [ ]:
#| default_exp policy/policy_gap_analyzer

In [ ]:
#| export
from app.policy.helper.llm_functions import init_azure_llm
from app.policy.helper.prompts import policy_gap_prompt_template, policy_gap_output
import concurrent.futures
import json

In [ ]:
#| export
def generate_policy_gap(
    llm, # LLM object
    domain, # single domain object of given standard
    policy, # entire policy object
    standard_name # standard name
):
    ONE_DOMAIN = domain
    ONE_DOMAIN['standard_name']=standard_name
    ONE_DOMAIN = f"""
        ```json 
        {str(ONE_DOMAIN)}
        ```"""
    input_policy_str = f"""
        ```json
        {str(policy)}
        ```"""
    
    prompt = policy_gap_prompt_template.format(input_policy=input_policy_str,input_standard=ONE_DOMAIN,output_str=policy_gap_output)
    response = llm.bind(response_format={"type": "json_object"}).invoke(prompt)
    return response

In [ ]:
#| export
def policy_gap_summary(
    llm, # LLM object
    policy_gap_resp # LLM responses of all domain
):
    # Combine gap_analysis descriptions
    combined_description = " ".join([json.loads(resp.content)['gap_analysis']['description'] for resp in policy_gap_resp])

    # Summarize the combined description using llm
    summary_response = llm.invoke(f"Summarize the following text: {combined_description}")
    summary_description = summary_response.content.strip()

    # Combine missing_controls and matching_controls
    combined_missing_controls = []
    combined_matching_controls = []

    for resp in policy_gap_resp:
        content = json.loads(resp.content)
        combined_missing_controls.extend(content['controls_comparison']['missing_controls'])
        combined_matching_controls.extend(content['controls_comparison']['matching_controls'])

    # Create the combined output
    combined_output = {
        "status": "success",
        "gap_analysis": {
            "description": summary_description
        },
        "controls_comparison": {
            "missing_controls": combined_missing_controls,
            "matching_controls": combined_matching_controls
        }
    }
    return combined_output

In [ ]:
#| export
def policy_gap_processor(input_data): # input data [Policy, Standard]
    policy_gap_resp = []
    print('policy gap analyzer api process started')
    llm = init_azure_llm()
    [input_policy, input_standard] = input_data
    standard_name = input_standard['standard_name']
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(generate_policy_gap, llm,domain,input_policy,standard_name) for domain in input_standard['domains']]
        for future in concurrent.futures.as_completed(futures):
            policy_gap_resp.append(future.result())

    combined_output = policy_gap_summary(llm, policy_gap_resp)
    print('policy gap analyzer api process Completed')
    return combined_output

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()